In [1]:
# Import packages
import pandas as pd
import datetime as dt
from pandas_profiling import ProfileReport

In [2]:
import sqlite3
print (sqlite3.sqlite_version)

3.32.3


## 1. Read SQL files and create dataframes

In [3]:
def parse_sql(filename):

    data = open(filename, 'r').read()

    stmts = []

    DELIMITER = ';'

    stmt = ''

    tab ='\t'



    for lineno, line in enumerate(data):

        if not line.strip():

            continue

        if line.startswith('--'):

            continue

        if 'DELIMITER' in line:

            DELIMITER = line.split()[1]

            continue

        if (DELIMITER not in line):

            stmt += line.replace(DELIMITER, ';')

            continue

        if 'tab' in line:

            tab=line.split()[1]

            continue

        if stmt:

            stmt += line

            stmts.append(stmt.strip())

            stmt = ''

        else:

            stmts.append(line.strip())

    return stmts

### 1.1. Load and read Bookings.sql file, create a dataframe named Bookings

In [4]:
#parse the Booking.sql file.
parse_sql('Bookings.sql')

['--phpMyAdminSQLDump--version4.0.10deb1--http://www.phpmyadmin.net----Host:localhost--GenerationTime:Dec20,2020at02:10PM--Serverversion:5.5.43-0ubuntu0.14.04.1--PHPVersion:5.5.9-1ubuntu4.11SETSQL_MODE="NO_AUTO_VALUE_ON_ZERO";',
 'SETtime_zone="+00:00";',
 '/*!40101SET@OLD_CHARACTER_SET_CLIENT=@@CHARACTER_SET_CLIENT*/;',
 '/*!40101SET@OLD_CHARACTER_SET_RESULTS=@@CHARACTER_SET_RESULTS*/;',
 '/*!40101SET@OLD_COLLATION_CONNECTION=@@COLLATION_CONNECTION*/;',
 '/*!40101SETNAMESutf8*/;',
 "----Database:`country_club`----------------------------------------------------------------Tablestructurefortable`Bookings`--CREATETABLEIFNOTEXISTS`Bookings`(`bookid`int(4)NOTNULLDEFAULT'0',`facid`int(1)DEFAULTNULL,`memid`int(2)DEFAULTNULL,`starttime`varchar(19)DEFAULTNULL,`slots`int(2)DEFAULTNULL,PRIMARYKEY(`bookid`))ENGINE=InnoDBDEFAULTCHARSET=utf8;",
 "----Dumpingdatafortable`Bookings`--INSERTINTO`Bookings`(`bookid`,`facid`,`memid`,`starttime`,`slots`)VALUES(0,3,1,'2012-07-0311:00:00',2),(1,4,1,'2012-07

In [5]:
len(parse_sql('Booking.sql')[10])

3841

In [6]:
#Check out name of columns in the sql file and clean them
col_labels = parse_sql('Booking.sql')[10][21:65]
col_labels

'`bookid`,`facid`,`memid`,`starttime`,`slots`'

In [7]:
col_labels = col_labels.replace('`','')
col_labels

'bookid,facid,memid,starttime,slots'

In [8]:
col_labels = col_labels.split(',')
col_labels

['bookid', 'facid', 'memid', 'starttime', 'slots']

In [9]:
#Extract the values and check its type
type(parse_sql('Booking.sql')[10][72:3840])

str

In [10]:
parse_sql('Booking.sql')[10][72:141]

"(3934,4,13,'2012-09-2917:30:00',2),(3935,4,36,'2012-09-2918:30:00',2)"

##### Values are formated as a string of tuples.

In [11]:
#Convert string of tuples to a list of tuples
tpl = eval(parse_sql('Booking.sql')[10][72:3840])
tpl[0]

(3934, 4, 13, '2012-09-2917:30:00', 2)

In [12]:
lst = list(tpl)
lst[0:2]

[(3934, 4, 13, '2012-09-2917:30:00', 2),
 (3935, 4, 36, '2012-09-2918:30:00', 2)]

In [13]:
#Create a dataframe 
Bookings = pd.DataFrame(lst)
Bookings.columns = col_labels
Bookings.head()

,bookid,facid,memid,starttime,slots
0,3934,4,13,2012-09-2917:30:00,2
1,3935,4,36,2012-09-2918:30:00,2
2,3936,4,24,2012-09-2919:30:00,2
3,3937,5,0,2012-09-2912:30:00,2
4,3938,6,6,2012-09-2908:00:00,2


### 1.2. Load and read Facilities.sql file, create a dataframe named Facilities

In [14]:
#parse the Facilities.sql file.
parse_sql('Facilities.sql')

['--phpMyAdminSQLDump--version4.0.10deb1--http://www.phpmyadmin.net----Host:localhost--GenerationTime:Dec20,2020at05:46PM--Serverversion:5.5.43-0ubuntu0.14.04.1--PHPVersion:5.5.9-1ubuntu4.11SETSQL_MODE="NO_AUTO_VALUE_ON_ZERO";',
 'SETtime_zone="+00:00";',
 '/*!40101SET@OLD_CHARACTER_SET_CLIENT=@@CHARACTER_SET_CLIENT*/;',
 '/*!40101SET@OLD_CHARACTER_SET_RESULTS=@@CHARACTER_SET_RESULTS*/;',
 '/*!40101SET@OLD_COLLATION_CONNECTION=@@COLLATION_CONNECTION*/;',
 '/*!40101SETNAMESutf8*/;',
 "----Database:`country_club`----------------------------------------------------------------Tablestructurefortable`Facilities`--CREATETABLEIFNOTEXISTS`Facilities`(`facid`int(1)NOTNULLDEFAULT'0',`name`varchar(15)DEFAULTNULL,`membercost`decimal(2,1)DEFAULTNULL,`guestcost`decimal(3,1)DEFAULTNULL,`initialoutlay`int(5)DEFAULTNULL,`monthlymaintenance`int(4)DEFAULTNULL,PRIMARYKEY(`facid`))ENGINE=InnoDBDEFAULTCHARSET=utf8;",
 "----Dumpingdatafortable`Facilities`--INSERTINTO`Facilities`(`facid`,`name`,`membercost`,`

In [15]:
parse_sql('Facilities.sql')[7]

"----Dumpingdatafortable`Facilities`--INSERTINTO`Facilities`(`facid`,`name`,`membercost`,`guestcost`,`initialoutlay`,`monthlymaintenance`)VALUES(0,'TennisCourt1',5.0,25.0,10000,200),(1,'TennisCourt2',5.0,25.0,8000,200),(2,'BadmintonCourt',0.0,15.5,4000,50),(3,'TableTennis',0.0,5.0,320,10),(4,'MassageRoom1',9.9,80.0,4000,3000),(5,'MassageRoom2',9.9,80.0,4000,3000),(6,'SquashCourt',3.5,17.5,5000,80),(7,'SnookerTable',0.0,5.0,450,15),(8,'PoolTable',0.0,5.0,400,15);"

In [16]:
len(parse_sql('Facilities.sql')[7])

465

In [17]:
#Check out name of columns in the sql file and clean them
col_labels_fac = parse_sql('Facilities.sql')[7][60:136]
col_labels_fac

'`facid`,`name`,`membercost`,`guestcost`,`initialoutlay`,`monthlymaintenance`'

In [18]:
col_labels_fac = col_labels_fac.replace('`','')
col_labels_fac

'facid,name,membercost,guestcost,initialoutlay,monthlymaintenance'

In [19]:
type(col_labels_fac)

str

In [20]:
col_labels_fac = col_labels_fac.split(',')
col_labels_fac

['facid',
 'name',
 'membercost',
 'guestcost',
 'initialoutlay',
 'monthlymaintenance']

In [21]:
#Extract the values and check its type
type(parse_sql('Facilities.sql')[7][143:465])

str

In [22]:
parse_sql('Facilities.sql')[7][143:465]

"(0,'TennisCourt1',5.0,25.0,10000,200),(1,'TennisCourt2',5.0,25.0,8000,200),(2,'BadmintonCourt',0.0,15.5,4000,50),(3,'TableTennis',0.0,5.0,320,10),(4,'MassageRoom1',9.9,80.0,4000,3000),(5,'MassageRoom2',9.9,80.0,4000,3000),(6,'SquashCourt',3.5,17.5,5000,80),(7,'SnookerTable',0.0,5.0,450,15),(8,'PoolTable',0.0,5.0,400,15);"

##### Values are formated as a string of tuples.

In [23]:
parse_sql('Facilities.sql')[7][143:]

"(0,'TennisCourt1',5.0,25.0,10000,200),(1,'TennisCourt2',5.0,25.0,8000,200),(2,'BadmintonCourt',0.0,15.5,4000,50),(3,'TableTennis',0.0,5.0,320,10),(4,'MassageRoom1',9.9,80.0,4000,3000),(5,'MassageRoom2',9.9,80.0,4000,3000),(6,'SquashCourt',3.5,17.5,5000,80),(7,'SnookerTable',0.0,5.0,450,15),(8,'PoolTable',0.0,5.0,400,15);"

In [24]:
#Convert string of tuples to a list of tuples
tpl_fac = eval(parse_sql('Facilities.sql')[7][143:464])
tpl_fac

((0, 'TennisCourt1', 5.0, 25.0, 10000, 200),
 (1, 'TennisCourt2', 5.0, 25.0, 8000, 200),
 (2, 'BadmintonCourt', 0.0, 15.5, 4000, 50),
 (3, 'TableTennis', 0.0, 5.0, 320, 10),
 (4, 'MassageRoom1', 9.9, 80.0, 4000, 3000),
 (5, 'MassageRoom2', 9.9, 80.0, 4000, 3000),
 (6, 'SquashCourt', 3.5, 17.5, 5000, 80),
 (7, 'SnookerTable', 0.0, 5.0, 450, 15),
 (8, 'PoolTable', 0.0, 5.0, 400, 15))

In [25]:
lst_fac = list(tpl_fac)
lst_fac[0:2]


[(0, 'TennisCourt1', 5.0, 25.0, 10000, 200),
 (1, 'TennisCourt2', 5.0, 25.0, 8000, 200)]

In [26]:
#Create a dataframe 
Facilities = pd.DataFrame(lst_fac)
Facilities.columns = col_labels_fac
Facilities.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,TennisCourt1,5.0,25.0,10000,200
1,1,TennisCourt2,5.0,25.0,8000,200
2,2,BadmintonCourt,0.0,15.5,4000,50
3,3,TableTennis,0.0,5.0,320,10
4,4,MassageRoom1,9.9,80.0,4000,3000


### 1.3. Load and read Members.sql file, create a dataframe named Members

In [27]:
#parse the Members.sql file.
parse_sql('Members.sql')

['--phpMyAdminSQLDump--version4.0.10deb1--http://www.phpmyadmin.net----Host:localhost--GenerationTime:Dec20,2020at05:46PM--Serverversion:5.5.43-0ubuntu0.14.04.1--PHPVersion:5.5.9-1ubuntu4.11SETSQL_MODE="NO_AUTO_VALUE_ON_ZERO";',
 'SETtime_zone="+00:00";',
 '/*!40101SET@OLD_CHARACTER_SET_CLIENT=@@CHARACTER_SET_CLIENT*/;',
 '/*!40101SET@OLD_CHARACTER_SET_RESULTS=@@CHARACTER_SET_RESULTS*/;',
 '/*!40101SET@OLD_COLLATION_CONNECTION=@@COLLATION_CONNECTION*/;',
 '/*!40101SETNAMESutf8*/;',
 "----Database:`country_club`----------------------------------------------------------------Tablestructurefortable`Members`--CREATETABLEIFNOTEXISTS`Members`(`memid`int(2)NOTNULLDEFAULT'0',`surname`varchar(17)DEFAULTNULL,`firstname`varchar(9)DEFAULTNULL,`address`varchar(39)DEFAULTNULL,`zipcode`int(5)DEFAULTNULL,`telephone`varchar(14)DEFAULTNULL,`recommendedby`varchar(2)DEFAULTNULL,`joindate`varchar(19)DEFAULTNULL,PRIMARYKEY(`memid`))ENGINE=InnoDBDEFAULTCHARSET=utf8;",
 "----Dumpingdatafortable`Members`--INSE

In [28]:
len(parse_sql('Members.sql')[7])

3136

In [29]:
#Check out name of columns in the sql file and clean them
col_labels_mem = parse_sql('Members.sql')[7][54:142]
col_labels_mem

'`memid`,`surname`,`firstname`,`address`,`zipcode`,`telephone`,`recommendedby`,`joindate`'

In [30]:
col_labels_mem = col_labels_mem.replace('`','')
col_labels_mem = col_labels_mem.split(',')


In [31]:
col_labels_mem

['memid',
 'surname',
 'firstname',
 'address',
 'zipcode',
 'telephone',
 'recommendedby',
 'joindate']

In [32]:
parse_sql('Members.sql')[7][149:3135]

"(0,'GUEST','GUEST','GUEST',0,'(000)000-0000','','2012-07-0100:00:00'),(1,'Smith','Darren','8BloomsburyClose,Boston',4321,'555-555-5555','','2012-07-0212:02:05'),(2,'Smith','Tracy','8BloomsburyClose,NewYork',4321,'555-555-5555','','2012-07-0212:08:23'),(3,'Rownam','Tim','23HighwayWay,Boston',23423,'(844)693-0723','','2012-07-0309:32:15'),(4,'Joplette','Janice','20CrossingRoad,NewYork',234,'(833)942-4710','1','2012-07-0310:25:05'),(5,'Butters','Gerald','1065HuntingdonAvenue,Boston',56754,'(844)078-4130','1','2012-07-0910:44:09'),(6,'Tracy','Burton','3TunisiaDrive,Boston',45678,'(822)354-9973','','2012-07-1508:52:55'),(7,'Dare','Nancy','6HuntingLodgeWay,Boston',10383,'(833)776-4001','4','2012-07-2508:59:12'),(8,'Boothe','Tim','3BloomsburyClose,Reading,00234',234,'(811)433-2547','3','2012-07-2516:02:35'),(9,'Stibbons','Ponder','5DragonsWay,Winchester',87630,'(833)160-3900','6','2012-07-2517:09:05'),(10,'Owen','Charles','52CheshireGrove,Winchester,28563',28563,'(855)542-5251','1','2012-08-

In [33]:
#Convert string of tuples to a list of tuples
tpl_fac_mem = eval(parse_sql('Members.sql')[7][149:3135])
tpl_fac_mem[0]

(0, 'GUEST', 'GUEST', 'GUEST', 0, '(000)000-0000', '', '2012-07-0100:00:00')

In [34]:
lst_mem = list(tpl_fac_mem)
lst_mem[0:2]

[(0, 'GUEST', 'GUEST', 'GUEST', 0, '(000)000-0000', '', '2012-07-0100:00:00'),
 (1,
  'Smith',
  'Darren',
  '8BloomsburyClose,Boston',
  4321,
  '555-555-5555',
  '',
  '2012-07-0212:02:05')]

In [35]:
#Create a dataframe 
Members = pd.DataFrame(lst_mem)
Members.columns = col_labels_mem
Members.head()

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000)000-0000,,2012-07-0100:00:00
1,1,Smith,Darren,"8BloomsburyClose,Boston",4321,555-555-5555,,2012-07-0212:02:05
2,2,Smith,Tracy,"8BloomsburyClose,NewYork",4321,555-555-5555,,2012-07-0212:08:23
3,3,Rownam,Tim,"23HighwayWay,Boston",23423,(844)693-0723,,2012-07-0309:32:15
4,4,Joplette,Janice,"20CrossingRoad,NewYork",234,(833)942-4710,1,2012-07-0310:25:05


# 2. Questions 

## 2.1. Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! 

In [36]:
# Range of time that facilities booked
(Bookings.starttime.min() , Bookings.starttime.max())

('2012-09-2908:00:00', '2012-09-3019:30:00')

Facilities were booked for 2 days.

In [37]:
profile = ProfileReport(Bookings, minimal=True)
profile.to_widgets()

In [38]:
#Drop bookid and starttime columns and add a new categorical column named membership with two possibe values.
df = Bookings.drop(columns= ['bookid','starttime'])
df['membership'] =''
df.loc[df['memid'] == 0 ,'membership'] = 'guest'
df.loc[df['memid'] != 0 ,'membership'] = 'member'
summary = df.groupby(['facid', 'membership'])['slots'].sum()
summary.head()

facid  membership
0      guest          9
       member        15
1      guest          3
       member        18
2      member        24
Name: slots, dtype: int64

In [39]:
print(((df.facid == 2) & (df.memid == 0)).sum())
print(((df.facid == 8) & (df.memid == 0)).sum())
print(((df.facid == 5) & (df.memid != 0)).sum())

0
0
0


### BadmintonCourt and PoolTable were only used by members. MassageRoom2 was only used by guests.

In [40]:
# Convert the multiIndex Series to a DataFrame by unstacking the inner index of membership.
df_small = summary.unstack(level=1)
df_small.columns = ['guest_slots', 'member_slots']
df_small = df_small.reset_index()
df_small

,facid,guest_slots,member_slots
0,0,9.0,15.0
1,1,3.0,18.0
2,2,NaN,24.0
3,3,2.0,10.0
4,4,6.0,20.0
5,5,6.0,NaN
6,6,24.0,14.0
7,7,2.0,30.0
8,8,NaN,39.0


In [41]:
#Merge the df_small with Facilities.
large_df = pd.merge(df_small, Facilities, on='facid')

#Calculate the maintenance cost of each facilities for 2 days.
large_df['monthlymaintenance'] = 2 * large_df['monthlymaintenance']/30

#Calculate the revenue generated by each facilities.
large_df['revenue'] = (large_df['guest_slots'] * large_df['guestcost']) 
+ (large_df['member_slots'] * large_df['membercost']) 
- large_df['monthlymaintenance']

large_df = large_df[large_df.revenue < 1000]
large_df

,facid,guest_slots,member_slots,name,membercost,guestcost,initialoutlay,monthlymaintenance,revenue
0,0,9.0,15.0,TennisCourt1,5.0,25.0,10000,13.333333,225.0
1,1,3.0,18.0,TennisCourt2,5.0,25.0,8000,13.333333,75.0
3,3,2.0,10.0,TableTennis,0.0,5.0,320,0.666667,10.0
4,4,6.0,20.0,MassageRoom1,9.9,80.0,4000,200.000000,480.0
5,5,6.0,NaN,MassageRoom2,9.9,80.0,4000,200.000000,480.0
6,6,24.0,14.0,SquashCourt,3.5,17.5,5000,5.333333,420.0
7,7,2.0,30.0,SnookerTable,0.0,5.0,450,1.000000,10.0


In [42]:
#list of facilities' names.
Facilities.name

0      TennisCourt1
1      TennisCourt2
2    BadmintonCourt
3       TableTennis
4      MassageRoom1
5      MassageRoom2
6       SquashCourt
7      SnookerTable
8         PoolTable
Name: name, dtype: object

In [43]:
#List of facilities that generated revenue less than 1000 dollars for the dates '2012-09-29' and '2012-09-30'.   
large_df [['name','revenue']].sort_values(by='revenue', ascending =False)

,name,revenue
4,MassageRoom1,480.0
5,MassageRoom2,480.0
6,SquashCourt,420.0
0,TennisCourt1,225.0
1,TennisCourt2,75.0
3,TableTennis,10.0
7,SnookerTable,10.0


### SnookerTable and TableTennis had least monthly revenue while least revenue generated by PoolTable was more than 1000 dollars.

## 2.2.Produce a report of members and who recommended them in alphabetic surname,firstname order

In [44]:
df_mem = Members[['memid','surname','firstname','recommendedby']]
df_mem = df_mem[df_mem['recommendedby'] != '']
df_mem.head()

,memid,surname,firstname,recommendedby
4,4,Joplette,Janice,1
5,5,Butters,Gerald,1
7,7,Dare,Nancy,4
8,8,Boothe,Tim,3
9,9,Stibbons,Ponder,6


In [45]:
#df_mem.fullname = pd.concat([df_mem.memid, df_mem.surname, df_mem.firstname],axis=1)
cols= ['surname','firstname']
df_mem['surname_firstname'] = df_mem [cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
df_mem = df_mem.drop(columns = ['memid','surname','firstname'])
df_mem.head()

,recommendedby,surname_firstname
4,1,Joplette Janice
5,1,Butters Gerald
7,4,Dare Nancy
8,3,Boothe Tim
9,6,Stibbons Ponder


In [46]:
#Convert type of column of 'recommendedby' as object to integer and sort dataframe.
df_mem['recommendedby'] = df_mem['recommendedby'].astype(str).astype(int)

In [47]:
#Find the name of references by merging.
id_name = Members[['memid','surname','firstname']]
merged_df = pd.merge(left= df_mem, right= id_name , left_on = 'recommendedby', right_on= 'memid')
merged_df.head()

,recommendedby,surname_firstname,memid,surname,firstname
0,1,Joplette Janice,1,Smith,Darren
1,1,Butters Gerald,1,Smith,Darren
2,1,Owen Charles,1,Smith,Darren
3,1,Smith Jack,1,Smith,Darren
4,1,Mackenzie Anna,1,Smith,Darren


In [48]:
#Join surname and first name of references, sort the name of references alphabetically.
merged_df['reference_name'] = merged_df[['surname','firstname']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
merged_df['reference_memid'] = merged_df['recommendedby']
merged_df = merged_df[['surname_firstname', 'reference_memid', 'reference_name']]
merged_df.sort_values('reference_name', inplace=True)
merged_df

,surname_firstname,reference_memid,reference_name
15,Sarwin Ramnaresh,15,Bader Florence
14,Coplin Joan,16,Baker Timothy
13,Genting Matthew,5,Butters Gerald
12,Pinker David,13,Farrell Jemima
11,Baker Timothy,13,Farrell Jemima
17,Rumney Henrietta,20,Genting Matthew
16,Jones Douglas,11,Jones David
5,Dare Nancy,4,Joplette Janice
6,Jones David,4,Joplette Janice
21,Hunt John,30,Purview Millicent


## 2.3. Find the facilities with their usage by member, but not guests

In [49]:
#Merge Bookings and Facilities on facid to extract facilities that only members used.
fac_use = Bookings[['facid','memid']]
fac_use = fac_use[fac_use.memid != 0]
df_fac = pd.merge(left= fac_use, right= Facilities, left_on= 'facid', right_on= 'facid' )

#Create a dataframe including member id and their names.
id_name = Members[['memid','surname','firstname']]
id_name['surname_firstname'] = id_name [['surname','firstname']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
id_name = id_name.drop(columns=['surname','firstname'])

#Merge two created dataframes to find that each facility used by which members.
fac_merged = pd.merge(left= df_fac, right= id_name , left_on = 'memid', right_on= 'memid')
fac_merged = fac_merged[['facid','name','memid','surname_firstname']]
fac_merged.head()

<ipython-input-49-b6c7fff2d75b>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_name['surname_firstname'] = id_name [['surname','firstname']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)


,facid,name,memid,surname_firstname
0,4,MassageRoom1,13,Farrell Jemima
1,4,MassageRoom1,13,Farrell Jemima
2,4,MassageRoom1,36,Crumpet Erica
3,0,TennisCourt1,36,Crumpet Erica
4,3,TableTennis,36,Crumpet Erica


In [50]:
#Check the members id used each facilities.
group = fac_merged.groupby('name')
df2 = group.apply(lambda x: x['memid'].unique())
df2

name
BadmintonCourt                                   [35, 5, 29, 17, 1]
MassageRoom1                             [13, 36, 24, 16, 20, 4, 3]
PoolTable         [16, 20, 4, 3, 12, 15, 33, 21, 29, 28, 2, 26, 17]
SnookerTable                              [24, 4, 27, 8, 15, 33, 5]
SquashCourt                                     [24, 6, 12, 27, 35]
TableTennis                                          [36, 24, 3, 1]
TennisCourt1                                        [36, 24, 4, 35]
TennisCourt2                                            [8, 10, 11]
dtype: object

### PoolTable had the highest number of users and TennisCourt2 had the fewest number of users.

In [51]:
# Check the name of members used each of facilities.
fac_merged= fac_merged.set_index(['facid','name'])
fac_merged

memid surname_firstname
facid name                                   
4     MassageRoom1       13    Farrell Jemima
      MassageRoom1       13    Farrell Jemima
      MassageRoom1       36     Crumpet Erica
0     TennisCourt1       36     Crumpet Erica
3     TableTennis        36     Crumpet Erica
...                     ...               ...
1     TennisCourt2       10      Owen Charles
      TennisCourt2       11       Jones David
2     BadmintonCourt      1      Smith Darren
      BadmintonCourt      1      Smith Darren
3     TableTennis         1      Smith Darren

[88 rows x 2 columns]

## 2.4. Find the facilities usage by month, but not guests

In [52]:
fac_month = Members[['memid', 'surname', 'firstname','joindate']]
fac_month = fac_month[fac_month.memid != 0]
fac_month.head()

,memid,surname,firstname,joindate
1,1,Smith,Darren,2012-07-0212:02:05
2,2,Smith,Tracy,2012-07-0212:08:23
3,3,Rownam,Tim,2012-07-0309:32:15
4,4,Joplette,Janice,2012-07-0310:25:05
5,5,Butters,Gerald,2012-07-0910:44:09


In [53]:
#Check joindate infac_month dataframe.
fac_month.joindate = fac_month.joindate.astype(str)
fac_month.joindate.head()

1    2012-07-0212:02:05
2    2012-07-0212:08:23
3    2012-07-0309:32:15
4    2012-07-0310:25:05
5    2012-07-0910:44:09
Name: joindate, dtype: object

##### unknown format.

In [54]:
fac_month.joindate = pd.to_datetime(fac_month.joindate.str[:-8])
fac_month.joindate.head()

1   2012-07-02
2   2012-07-02
3   2012-07-03
4   2012-07-03
5   2012-07-09
Name: joindate, dtype: datetime64[ns]

In [55]:
fac_month['join_month'] = fac_month.joindate.dt.month
fac_month.head()

,memid,surname,firstname,joindate,join_month
1,1,Smith,Darren,2012-07-02,7
2,2,Smith,Tracy,2012-07-02,7
3,3,Rownam,Tim,2012-07-03,7
4,4,Joplette,Janice,2012-07-03,7
5,5,Butters,Gerald,2012-07-09,7


In [56]:
fac_month_merged = pd.merge(left=fac_month, right= Bookings, left_on= 'memid', right_on= 'memid')
fac_month_merged = fac_month_merged[['memid','join_month', 'facid']]
fac_month_merged.head()

,memid,join_month,facid
0,1,7,2
1,1,7,2
2,1,7,3
3,2,7,8
4,3,7,8


In [57]:
month = fac_month_merged.groupby(['join_month','facid'])['memid'].count()
month_df = month.unstack(level=0)
merged_month = pd.merge(left=month_df, right= Facilities, left_on= 'facid', right_on='facid')
merged_month= merged_month[['facid','name',7, 8, 9]]
merged_month.columns= ['facid','name','Jul', 'Aug','Sep']
merged_month

,facid,name,Jul,Aug,Sep
0,0,TennisCourt1,1.0,NaN,4.0
1,1,TennisCourt2,2.0,3.0,NaN
2,2,BadmintonCourt,3.0,1.0,3.0
3,3,TableTennis,2.0,NaN,3.0
4,4,MassageRoom1,3.0,5.0,2.0
5,6,SquashCourt,1.0,2.0,4.0
6,7,SnookerTable,6.0,2.0,7.0
7,8,PoolTable,7.0,12.0,15.0


### In July, all facilities were used and SquashCourt was used most frequently.
### In Augest, TennisCourt1, TableTennis and MassageRoom2 were not used by any members but PoolTable was served by majority of members.
### Agin in Sep, PoolTable was most favor facility and only TennisCourt2 didn't serve any members.